# Data Science Championship - South Zone

> "Predicting the house rent of the given house details"

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [predict, machine, hack, the, house, rent, price, data, science, championship, south, zone, learning]
- hide: false

In [1]:
# Installing the modules

!pip install wurlitzer
!pip3 install category_encoders
!pip install tensorflow_decision_forests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 80 kB 5.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.8 MB 10.9 MB/s 
     |████████████████████████████████| 511.7 MB 5.7 kB/s 
     |████████████████████████████████| 5.8 MB 33.8 MB/s 
     |████████████████████████████████| 1.6 MB 41.5 MB/s 
     |████████████████████████████████| 438 kB 50.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully unins

In [2]:
# Required modules

import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import category_encoders as ce
import tensorflow_decision_forests as tfdf

from google.colab import drive
from matplotlib import pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# Config

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 12)
pd.set_option('display.max_columns', None)

In [4]:
# Mounting the drive

drive.mount('./mydrive')

Mounted at ./mydrive


In [5]:
# Moving files to workspace

shutil.copy('/content/mydrive/MyDrive/Machine Hack/Data Science Students Championship/train.csv', './train.csv')
shutil.copy('/content/mydrive/MyDrive/Machine Hack/Data Science Students Championship/test.csv', './test.csv')
shutil.copy('/content/mydrive/MyDrive/Machine Hack/Data Science Students Championship/submission.csv', './submission.csv')

'./submission.csv'

In [6]:
# Load the data

train = pd.read_csv('train.csv')
train.head()

,Property_ID,room,layout_type,property_type,locality,price,area,furnish_type,bathroom,city,parking_spaces,floor,pet_friendly,power_backup,washing_machine,air_conditioner,geyser/solar,security_deposit,CCTV/security,lift,neighbourhood
0,42208,3,BHK,Independent House,Palavakkam,33624,1312,Furnished,2,Chennai,1,1,1,0,0,1,0,302616,0,0,300
1,90879,1,BHK,Apartment,Manikonda,9655,1474,Unfurnished,2,Hyderabad,0,17,0,1,0,0,1,19310,0,1,1600
2,99943,3,BHK,Apartment,Jodhpur Park,23699,1837,Semi-Furnished,2,Kolkata,0,10,1,1,1,1,0,118495,0,1,3100
3,113926,1,BHK,Apartment,Chembur,6306,606,Unfurnished,1,Mumbai,0,18,0,0,0,0,0,37836,0,1,300
4,185438,1,BHK,Studio Apartment,Kalewadi Pandhapur Road,12008,498,Semi-Furnished,3,Pune,0,14,0,0,1,1,0,72048,0,1,0


In [7]:
# Inspect the data

train.info()
train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134683 entries, 0 to 134682
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Property_ID       134683 non-null  int64 
 1   room              134683 non-null  int64 
 2   layout_type       134683 non-null  object
 3   property_type     134683 non-null  object
 4   locality          134683 non-null  object
 5   price             134683 non-null  int64 
 6   area              134683 non-null  int64 
 7   furnish_type      134683 non-null  object
 8   bathroom          134683 non-null  int64 
 9   city              134683 non-null  object
 10  parking_spaces    134683 non-null  int64 
 11  floor             134683 non-null  int64 
 12  pet_friendly      134683 non-null  int64 
 13  power_backup      134683 non-null  int64 
 14  washing_machine   134683 non-null  int64 
 15  air_conditioner   134683 non-null  int64 
 16  geyser/solar      134683 non-null  int

,Property_ID,room,price,area,bathroom,parking_spaces,floor,pet_friendly,power_backup,washing_machine,air_conditioner,geyser/solar,security_deposit,CCTV/security,lift,neighbourhood
count,134683.000000,134683.000000,134683.000000,134683.000000,134683.000000,134683.000000,134683.000000,134683.000000,134683.000000,134683.000000,134683.000000,134683.000000,1.346830e+05,134683.000000,134683.000000,134683.000000
mean,96036.100777,2.029677,36690.033894,1480.388490,2.040488,0.534388,9.163087,0.527602,0.337051,0.472561,0.692626,0.440137,2.202480e+05,0.561838,0.595851,2033.024212
std,55565.228125,0.937308,62620.364025,1412.464718,0.867065,0.498818,5.957549,0.499239,0.472704,0.499248,0.461407,0.496405,4.204503e+05,0.496163,0.490728,1159.635981
min,2.000000,1.000000,1583.000000,81.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.180000e+03,0.000000,0.000000,0.000000
25%,47940.000000,1.000000,12035.500000,759.000000,1.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.580250e+04,0.000000,0.000000,1100.000000
50%,95950.000000,2.000000,20856.000000,1114.000000,2.000000,1.000000,9.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.142640e+05,1.000000,1.000000,2000.000000
75%,144194.500000,3.000000,36014.000000,1580.000000,2.000000,1.000000,15.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.207045e+05,1.000000,1.000000,3000.000000
max,192405.000000,5.000000,799325.000000,13942.000000,5.000000,1.000000,19.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.940780e+06,1.000000,1.000000,4000.000000


In [8]:
# Load the test data

test = pd.read_csv('./test.csv')
test.head()

,Property_ID,room,layout_type,property_type,locality,area,furnish_type,bathroom,city,parking_spaces,floor,pet_friendly,power_backup,washing_machine,air_conditioner,geyser/solar,security_deposit,CCTV/security,lift,neighbourhood,price
0,114342,2,BHK,Independent Floor,Palava,1347,Semi-Furnished,1,Mumbai,0,2,0,1,1,1,0,72624,1,0,900,NaN
1,88819,1,BHK,Independent House,Somajiguda,634,Semi-Furnished,3,Hyderabad,1,4,0,0,1,1,0,19656,0,0,2500,NaN
2,85623,1,BHK,Apartment,Toli Chowki,524,Unfurnished,1,Hyderabad,1,3,1,1,0,0,0,7500,0,0,3200,NaN
3,130856,3,BHK,Apartment,Thane West,1837,Unfurnished,5,Mumbai,1,9,1,0,0,0,1,137646,1,1,1200,NaN
4,40089,2,BHK,Apartment,Krishnarajapura,1208,Semi-Furnished,2,Bangalore,1,17,0,1,1,1,0,110898,0,1,1000,NaN


In [9]:
# Checking for the missing values

if train.isna().any().any():
    print(train.isna().any())
else:
    print("No Missing Values")

No Missing Values


## Feature Engineering

In [10]:
# Converting the categorical variables

train['layout_type'] = np.where(train['layout_type'] == 'BHK', 1, 0)
test['layout_type'] = np.where(test['layout_type'] == 'BHK', 1, 0)

for col in train.columns:
    if train[col].dtype == 'object':
        encoder = ce.cat_boost.CatBoostEncoder()

        train[col] = encoder.fit_transform(train[col], train['price'])
        test[col] = encoder.transform(test[col])

In [11]:
# Seperating out features and labels

X = train.drop(['Property_ID', 'price'], axis=1)
y = train['price']
X_test = test.drop(['Property_ID', 'price'], axis=1)

In [ ]:
# Scaling the train and test values

scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)
X_test_sclaed = scaler.transform(X_test)

scaler_label = MinMaxScaler(feature_range=(0, 1))
scaler_label.fit(y.values.reshape(-1, 1))
y_scaled = scaler_label.transform(y.values.reshape(-1, 1))

In [ ]:
# Train and test split

X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=88)

## Model Building

### Approach - 1

In [ ]:
# Model Definition

input_len = len(train.columns) - 2

model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(input_len,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='linear'),
])

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 64)                1280      
                                                                 
 dropout_21 (Dropout)        (None, 64)                0         
                                                                 
 dense_29 (Dense)            (None, 32)                2080      
                                                                 
 dropout_22 (Dropout)        (None, 32)                0         
                                                                 
 dense_30 (Dense)            (None, 8)                 264       
                                                                 
 dropout_23 (Dropout)        (None, 8)                 0         
                                                                 
 dense_31 (Dense)            (None, 1)                

In [ ]:
# Custom loss

def rmse(y_true, y_pred):
    return tf.math.sqrt(tf.keras.losses.mean_squared_error(y_true, y_pred))

In [ ]:
# Compiling the model

loss = rmse
optim = tf.keras.optimizers.Adam(learning_rate=0.0001)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint('custom_model_checkpoint.hdf5', save_best_only=True, custom_objects=[rmse])

model.compile(optimizer=optim, loss=loss, metrics=[tf.keras.losses.mean_squared_error])

In [ ]:
# Fitting the model

epochs = 20
batch_size = 64

model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=epochs, batch_size=batch_size, shuffle=True)

Epoch 1/20
1684/1684 [==============================] - 5s 2ms/step - loss: 0.0535 - mean_squared_error: 0.0110 - val_loss: 0.0308 - val_mean_squared_error: 0.0063
Epoch 2/20
1684/1684 [==============================] - 4s 2ms/step - loss: 0.0318 - mean_squared_error: 0.0066 - val_loss: 0.0308 - val_mean_squared_error: 0.0064
Epoch 3/20
1684/1684 [==============================] - 4s 2ms/step - loss: 0.0315 - mean_squared_error: 0.0066 - val_loss: 0.0308 - val_mean_squared_error: 0.0063
Epoch 4/20
1684/1684 [==============================] - 4s 2ms/step - loss: 0.0314 - mean_squared_error: 0.0066 - val_loss: 0.0308 - val_mean_squared_error: 0.0063
Epoch 5/20
1684/1684 [==============================] - 4s 2ms/step - loss: 0.0313 - mean_squared_error: 0.0066 - val_loss: 0.0306 - val_mean_squared_error: 0.0062
Epoch 6/20
1684/1684 [==============================] - 4s 2ms/step - loss: 0.0311 - mean_squared_error: 0.0066 - val_loss: 0.0303 - val_mean_squared_error: 0.0062
Epoch 7/20
1684/

In [ ]:
# Scoring the partitions

print(f"RMSE of Train is {mean_squared_error(scaler_label.inverse_transform(model.predict(X_train)), scaler_label.inverse_transform(y_train), squared=False)}")
print(f"RMSE of Valid is {mean_squared_error(scaler_label.inverse_transform(model.predict(X_valid)), scaler_label.inverse_transform(y_valid), squared=False)}")

RMSE of Train is 36337.83644900412
RMSE of Valid is 36745.74150641437


### Approach - 2

* Use TensorFlow Decision Forest in the prediction of house prices.

In [12]:
# Convert the dataset into a TensorFlow dataset.

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train, label="price", task=tfdf.keras.Task.REGRESSION)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test, label="price", task=tfdf.keras.Task.REGRESSION)

/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core.py:2542: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


In [13]:
# Train a Random Forest model.

model = tfdf.keras.RandomForestModel(task=tfdf.keras.Task.REGRESSION)
model.fit(train_ds)

# Summary of the model structure.
model.summary()

# Evaluate the model.
model.evaluate(test_ds)

Use /tmp/tmp03c19zqp as temporary training directory
Reading training dataset...
Training dataset read in 0:00:14.151155. Found 134683 examples.
Training model...
Model trained in 0:03:41.615186
Compiling model...
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.
Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: REGRESSION
Label: "__LABEL"

Input Features (20):
	CCTV/security
	Property_ID
	air_conditioner
	area
	bathroom
	city
	floor
	furnish_type
	geyser/solar
	layout_type
	lift
	locality
	neighbourhood
	parking_spaces
	pet_friendly
	power_backup
	property_type
	room
	security_deposit
	washing_machine

No weights

Variable Importance: MEAN_MIN_DEPTH:
    1.          "__LABEL" 13.945312 

0.0

In [14]:
# Generate the submission file

test = pd.DataFrame(index=range(X_test.shape[0]))
test['price'] = model.predict(test_ds)
test.to_csv('submission.csv', index=False)

58/58 [==============================] - 6s 103ms/step


With the approch-2, that is using tensorflow decision forest I got this (RMSE: 27126). 